In [1]:
import pickle
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
import matplotlib.pyplot as pl
from matplotlib import rcParams
from seaborn import PairGrid, heatmap, kdeplot
import cmocean.cm as cmo

In [2]:
% matplotlib inline
rcParams['axes.titlesize'] = 18
rcParams['xtick.labelsize'] = 16
rcParams['ytick.labelsize'] = 16
rcParams['font.size'] = 16

UsageError: Line magic function `%` not found.


In [3]:
df_pca = pd.read_pickle('../PickleJar/Interim/df_4_pca.pkl')
df_aph = pd.read_pickle('../PickleJar/Interim/df_4_ML4aph.pkl')

FileNotFoundError: [Errno 2] No such file or directory: '../PickleJar/Interim/df_4_pca.pkl'

In [18]:
df_pca.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 495 entries, 0 to 494
Data columns (total 6 columns):
PC1    494 non-null float64
PC2    494 non-null float64
PC3    494 non-null float64
PC4    494 non-null float64
PC5    494 non-null float64
PC6    494 non-null float64
dtypes: float64(6)
memory usage: 27.1 KB


In [11]:
df_aph.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 495 entries, 0 to 494
Data columns (total 32 columns):
oisst            495 non-null float64
etopo2           495 non-null float64
solz             495 non-null float64
sat_rho_rc412    495 non-null float64
sat_rho_rc443    495 non-null float64
sat_rho_rc490    495 non-null float64
sat_rho_rc510    495 non-null float64
sat_rho_rc555    495 non-null float64
sat_rho_rc670    495 non-null float64
datetime         495 non-null datetime64[ns]
lat              495 non-null float64
lon              495 non-null float64
aphy405          495 non-null float64
aphy411          495 non-null float64
aphy443          495 non-null float64
aphy455          495 non-null float64
aphy465          495 non-null float64
aphy489          495 non-null float64
aphy510          495 non-null float64
aphy520          495 non-null float64
aphy530          495 non-null float64
aphy550          495 non-null float64
aphy555          495 non-null float64
aphy560       

In [12]:
def make_features_from_dt(df, cleanup=True):
    yeardays = max(df.datetime.dt.dayofyear.max(), 365)
    dayminutes = 24 * 60
    doy_arg = 2 * np.pi * df.datetime.dt.dayofyear / yeardays
    mod_arg = 2 * np.pi * (df.datetime.dt.hour * 60 + df.datetime.dt.minute) / dayminutes
    df['sin_doy'] = np.sin(doy_arg)
    df['cos_doy'] = np.cos(doy_arg)
    df['sin_minofday'] = np.sin(mod_arg)
    df['cos_minofday'] = np.cos(mod_arg)
    if cleanup:
        df.drop('datetime', axis=1, inplace=True)
    
    
def make_features_from_latlon(df, cleanup=True):
    df['x'] = np.cos(np.deg2rad(df.lat)) * np.cos(np.deg2rad(df.lon))
    df['y'] = np.cos(np.deg2rad(df.lat)) * np.sin(np.deg2rad(df.lon))
    df['z'] = np.sin(np.deg2rad(df.lat))
    if cleanup:
        df.drop(['lat', 'lon'], axis=1, inplace=True)

def log_transform_feature(df, feature, cleanup=True):
    df['log10_' + feature] = np.log10(df[feature]+1e-6)
    if cleanup:
        df.drop(feature, axis=1, inplace=True)

In [13]:
satcols = df_aph.filter(regex='sat').columns
df_aph_nosat = df_aph.drop(satcols, axis=1)

In [14]:
log_transform_feature(df_aph, 'etopo2')
log_transform_feature(df_aph_nosat, 'etopo2')

In [15]:
make_features_from_dt(df_aph)
make_features_from_dt(df_aph_nosat)

In [16]:
make_features_from_latlon(df_aph)
make_features_from_latlon(df_aph_nosat)

In [17]:
df_aph.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 495 entries, 0 to 494
Data columns (total 36 columns):
oisst            495 non-null float64
solz             495 non-null float64
sat_rho_rc412    495 non-null float64
sat_rho_rc443    495 non-null float64
sat_rho_rc490    495 non-null float64
sat_rho_rc510    495 non-null float64
sat_rho_rc555    495 non-null float64
sat_rho_rc670    495 non-null float64
aphy405          495 non-null float64
aphy411          495 non-null float64
aphy443          495 non-null float64
aphy455          495 non-null float64
aphy465          495 non-null float64
aphy489          495 non-null float64
aphy510          495 non-null float64
aphy520          495 non-null float64
aphy530          495 non-null float64
aphy550          495 non-null float64
aphy555          495 non-null float64
aphy560          495 non-null float64
aphy565          495 non-null float64
aphy570          495 non-null float64
aphy590          495 non-null float64
aphy619          495 

In [19]:
cols =['sin_doy', 'cos_doy', 'sin_minofday', 'cos_minofday', 'x', 'y', 'z',
       'log10_etopo2', 'oisst', 'solz']
df_features = df_aph_nosat[cols]
df_features = pd.concat((df_features, df_pca), axis=1)

In [20]:
df_nosat = pd.concat((df_features, df_aph_nosat.filter(regex='aphy', axis=1)), axis=1)

In [21]:
df_features.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 495 entries, 0 to 494
Data columns (total 16 columns):
sin_doy         495 non-null float64
cos_doy         495 non-null float64
sin_minofday    495 non-null float64
cos_minofday    495 non-null float64
x               495 non-null float64
y               495 non-null float64
z               495 non-null float64
log10_etopo2    495 non-null float64
oisst           495 non-null float64
solz            495 non-null float64
PC1             494 non-null float64
PC2             494 non-null float64
PC3             494 non-null float64
PC4             494 non-null float64
PC5             494 non-null float64
PC6             494 non-null float64
dtypes: float64(16)
memory usage: 65.7 KB


In [22]:
df_nosat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 495 entries, 0 to 494
Data columns (total 36 columns):
sin_doy         495 non-null float64
cos_doy         495 non-null float64
sin_minofday    495 non-null float64
cos_minofday    495 non-null float64
x               495 non-null float64
y               495 non-null float64
z               495 non-null float64
log10_etopo2    495 non-null float64
oisst           495 non-null float64
solz            495 non-null float64
PC1             494 non-null float64
PC2             494 non-null float64
PC3             494 non-null float64
PC4             494 non-null float64
PC5             494 non-null float64
PC6             494 non-null float64
aphy405         495 non-null float64
aphy411         495 non-null float64
aphy443         495 non-null float64
aphy455         495 non-null float64
aphy465         495 non-null float64
aphy489         495 non-null float64
aphy510         495 non-null float64
aphy520         495 non-null float64
aphy530

In [25]:
df_nosat[df_nosat.PC1.isna()]

,sin_doy,cos_doy,sin_minofday,cos_minofday,x,y,z,log10_etopo2,oisst,solz,...,aphy555,aphy560,aphy565,aphy570,aphy590,aphy619,aphy625,aphy665,aphy670,aphy683
174,-0.835925,-0.548843,-0.854912,0.518773,0.10113,-0.883359,0.457657,1.623249,29.9,20.3,...,0.00065,0.0005,0.00041,0.00035,0.0003,0.00047,0.00051,0.00143,0.00179,0.0011


In [30]:
df_aph.loc[174:174]

,oisst,solz,sat_rho_rc412,sat_rho_rc443,sat_rho_rc490,sat_rho_rc510,sat_rho_rc555,sat_rho_rc670,aphy405,aphy411,...,aphy670,aphy683,log10_etopo2,sin_doy,cos_doy,sin_minofday,cos_minofday,x,y,z
174,29.9,20.3,0.0,0.0,0.0,0.0,0.0,0.0,0.00981,0.01085,...,0.00179,0.0011,1.623249,-0.835925,-0.548843,-0.854912,0.518773,0.10113,-0.883359,0.457657


In [31]:
df_nosat.to_pickle('./pickleJar/DevelopmentalDataSets/df_5_APHY_pc.pkl')
df_aph.to_pickle('./pickleJar/DevelopmentalDataSets/df_5_APHY_sat')